In [ ]:
import iio

def main():
	ctx = iio.Context()

	print ('Library version: {}.{} (git tag: {})'.format(iio.version[0], iio.version[1], iio.version[2].decode()))

	print ('IIO context created: %s' % ctx.name)
	print ('Backend version: {}.{} (git tag: {})'.format(ctx.version[0], ctx.version[1], ctx.version[2].decode()))
	print ('Backend description string: {}'.format(ctx.description.decode()))

	print ('IIO context has %u devices:' % len(ctx.devices))

	for dev in ctx.devices:
		print ('\t{}: {}'.format(dev.id.decode(), dev.name.decode()))

		if dev is iio.Trigger:
			print ('Found trigger! Rate: {} Hz'.format(dev.frequency))

		print ('\t\t{} channels found:'.format(len(dev.channels)))

		for chn in dev.channels:
			print ('\t\t\t{}: {} ({})'.format(chn.id.decode(), chn.name.decode() if chn.name else "", 'output' if chn.output else 'input'))

			if len(chn.attrs) != 0:
				print ('\t\t\t{} channel-specific attributes found:'.format(len(chn.attrs)))

			for attr in chn.attrs:
				print ('\t\t\t\t{}, value: {}'.format(attr.decode(), chn.attrs[attr].value.decode()))

		if len(dev.attrs) != 0:
			print ('\t\t{} device-specific attributes found:'.format(len(dev.attrs)))

		for attr in dev.attrs:
			print ('\t\t\t{}, value: '.format(attr.decode(), dev.attrs[attr].value.decode()))

		if len(dev.debug_attrs) != 0:
			print ('\t\t{} debug attributes found:'.format(len(dev.debug_attrs)))

		for attr in dev.debug_attrs:
			print ('\t\t\t{}, value: '.format(attr.decode(), dev.debug_attrs[attr].value.decode()))

if __name__ == '__main__':
	main()

In [ ]:
ctx = iio.Context()

print ('IIO context has %u devices:' % len(ctx.devices))

dev = ctx.devices[2]

for chn in dev.channels:
    print ('{}: {} ({})'.format(chn.id.decode(), chn.name.decode() if chn.name else "", 'o' if chn.output else 'i'))
    raw    = chn.attrs[b'raw'].value.decode()
    scale  = chn.attrs[b'scale'].value.decode()
    if len(chn.attrs) == 3:
        offset = chn.attrs[b'offset'].value.decode()
        print ('\t({}+{})*{}/1000={}'.format(raw, offset, scale, (int(raw)+int(offset))*float(scale)/1000))
    else:
        print ('\t{}*{}/1000={}'.format(raw, scale, int(raw)*float(scale)/1000))

In [ ]:
import iio
ctx = iio.Context()
trg = ctx.devices[1]
dev = ctx.devices[3]
chn = dev.channels[3]
chn.scan_element
chn.enabled = True
dev.trigger = trg

https://github.com/jeras/libiio/blob/master/examples/dummy-iiostream.c

https://github.com/jeras/libiio/blob/master/bindings/python/iio.py

https://github.com/analogdevicesinc/libiio/pull/88

In [ ]:
buf = iio.Buffer(dev, 64, cyclic=False)

In [ ]:
buf